# Prerequesites

In [1]:
!pip install librosa soundfile -q
!pip install --upgrade accelerate transformers datasets evaluate jiwer wandb gradio -q
!pip install -U bitsandbytes -q
!pip install -q peft
!pip install -q gradio
# !pip install -q git+https://github.com/huggingface/peft.git@main


[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
DATASET_NAME = "gauravparajuli/slr43_slr54_slr143"
MODEL_NAME = "openai/whisper-large-v3-turbo"
LANGUAGE = 'Nepali'
TASK = 'transcribe'
TEST_SIZE = 0.1 # 10%

SEED=7

In [3]:
LORA_R = 32
LORA_ALPHA = 64

# Load Model, Processor, Dataset

In [4]:
from datasets import Audio, load_dataset
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
import torch
from transformers import (
    WhisperForConditionalGeneration,
    WhisperProcessor,
    BitsAndBytesConfig
)

dataset = load_dataset(DATASET_NAME, cache_dir='hf_cache') # sampling rate is 48 Khz for this dataset
dataset = dataset['train'].train_test_split(test_size=TEST_SIZE, seed=SEED)

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    cache_dir='hf_cache',
    quantization_config=bnb_config,
    device_map='auto'
)

# override generation tokens
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

processor = WhisperProcessor.from_pretrained(MODEL_NAME, task=TASK, language=LANGUAGE)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# Configure LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[
        "q_proj",
        "v_proj",
    ],
    lora_dropout=0.05,
    bias="none"
)

# peft model
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

README.md:   0%|          | 0.00/441 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/19 [00:00<?, ?it/s]

train-00000-of-00019.parquet:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

train-00001-of-00019.parquet:   0%|          | 0.00/387M [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

trainable params: 6,553,600 || all params: 815,431,680 || trainable%: 0.8037


# Dataset Preprocessing, Error Metric, Data Collator

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'text'],
        num_rows: 130119
    })
    test: Dataset({
        features: ['audio', 'text'],
        num_rows: 14458
    })
})

In [6]:
from datasets import Audio

sampling_rate = processor.feature_extractor.sampling_rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate)) # audio will only be resampled when it is loaded (on the fly)

In [ ]:
def prepare_dataset(example):
  # load and resample audio data from 48khz to 16khz
  audio = example['audio']

  # compute log-mel input features from input audio array
  example['input_features'] = processor.feature_extractor(
      audio['array'],
      sampling_rate=audio['sampling_rate'],
  ).input_features[0]

  # encode target text to label data
  example['labels'] = processor.tokenizer(example['text']).input_ids

  return example

# Process dataset
dataset = dataset.map(
    prepare_dataset,
    remove_columns=dataset.column_names["train"],
    num_proc=1
)

Map:   0%|          | 0/130119 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 1651
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 413
    })
})

In [5]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union


@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        # print(features)
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [6]:
import evaluate

wer_metric = evaluate.load("wer")
cer_metric = evaluate.load('cer')

# from transformers.models.whisper.english_normalizer import BasicTextNormalizer

# normalizer = BasicTextNormalizer()

def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = processor.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.batch_decode(label_ids, skip_special_tokens=True)

    # compute orthographic wer
    wer = 100 * wer_metric.compute(predictions=pred_str, references=label_str)
    cer = 100 * cer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer, "cer": cer}

# Training

In [ ]:
from transformers import Seq2SeqTrainingArguments

NUM_EPOCHS = 3
BATCH_SIZE = 16
OUTPUT_DIR = 'nepali-whisper-large-v3-turbo'
LEARNING_RATE = 1e-3

training_args = Seq2SeqTrainingArguments(
    output_dir="whisper-nepali-qlora",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,
    # eval_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    num_train_epochs=NUM_EPOCHS,                # Set to 5 epochs
    warmup_ratio=0.1,                          # Warm up over 10% of total steps
    evaluation_strategy="epoch",               # Evaluate every epoch
    save_strategy="epoch",                     # Save checkpoint every epoch
    logging_steps=25,
    report_to=["wandb"],

    # metric_for_best_model="wer",
    # greater_is_better=False,

    push_to_hub=False,

    # required as Peft Model's forward doesn't have the signature of the base model's forward
    remove_unused_columns=False,
    label_names=["labels"],

    fp16=True,                                # Mixed precision training
    seed=42,                                  # For reproducibility
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR
import os


class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    data_collator=data_collator,
    # compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks=[SavePeftModelCallback],
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

<ipython-input-11-c9b424ad3fbc>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.259800,0.283067
2,0.217200,0.248800
3,0.160600,0.234134


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentran

TrainOutput(global_step=312, training_loss=0.21499950687090555, metrics={'train_runtime': 2678.7192, 'train_samples_per_second': 1.849, 'train_steps_per_second': 0.116, 'total_flos': 8.51941085478912e+18, 'train_loss': 0.21499950687090555, 'epoch': 3.0})

## Push to hub

In [ ]:
peft_model_id = 'gauravparajuli/' + f'{MODEL_NAME}-LORA-Nepali'.replace('/', '-')
model.push_to_hub(peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gauravparajuli/openai-whisper-large-v3-turbo-LORA-Nepali/commit/90b67490c0a465c9f33f7a8363a013fc13a56653', commit_message='Upload model', commit_description='', oid='90b67490c0a465c9f33f7a8363a013fc13a56653', pr_url=None, repo_url=RepoUrl('https://huggingface.co/gauravparajuli/openai-whisper-large-v3-turbo-LORA-Nepali', endpoint='https://huggingface.co', repo_type='model', repo_id='gauravparajuli/openai-whisper-large-v3-turbo-LORA-Nepali'), pr_revision=None, pr_num=None)

# Evaluation

In [10]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer

peft_model_id = 'gauravparajuli/openai-whisper-large-v3-turbo-LORA-Nepali'
peft_config = PeftConfig.from_pretrained(peft_model_id)

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    cache_dir='hf_cache',
    quantization_config=bnb_config,
    device_map='auto'
)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/26.2M [00:00<?, ?B/s]

In [7]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
import gc

BATCH_SIZE = 32

test_dataloader = DataLoader(dataset['test'], BATCH_SIZE, collate_fn=data_collator)

model.eval()
for step, batch in enumerate(tqdm(test_dataloader)):
  with torch.cuda.amp.autocast():
    with torch.no_grad():
      generated_tokens = model.generate(
          input_features=batch['input_features'].to('cuda'),
          decoder_input_ids=batch['labels'][:, :4].to('cuda'),
          # max_new_tokens=100,
      ).cpu().numpy()

      labels = batch['labels'].numpy()
      labels = np.where(labels != -100, labels, processor.tokenizer.pad_token_id)
      decoded_preds = processor.tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
      decoded_labels = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)
      wer_metric.add_batch(predictions=decoded_preds, references=decoded_labels)
      cer_metric.add_batch(predictions=decoded_preds, references=decoded_labels)

  del generated_tokens, labels, batch
  gc.collect()

wer = 100 * wer_metric.compute()
cer = 100 * cer_metric.compute()

print()
print(f'WER: {wer}')
print(f'CER: {cer}')

  0%|          | 0/13 [00:00<?, ?it/s]<ipython-input-7-b47e70d0aa62>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass 


WER: 83.35220838052095
CER: 24.487248824728542


# Using ASR pipeline

In [ ]:
from peft import PeftModel, PeftConfig
from transformers import WhisperForConditionalGeneration, Seq2SeqTrainer, AutomaticSpeechRecognitionPipeline

peft_model_id = 'gauravparajuli/openai-whisper-large-v3-turbo-LORA-Nepali'
peft_config = PeftConfig.from_pretrained(peft_model_id)

# Quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = WhisperForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    cache_dir='hf_cache',
    quantization_config=bnb_config,
    device_map='auto'
)
model = PeftModel.from_pretrained(model, peft_model_id)
processor = WhisperProcessor.from_pretrained(MODEL_NAME, task=TASK, language=LANGUAGE)
forced_decoder_ids = processor.get_decoder_prompt_ids(language=LANGUAGE, task=TASK)
pipe = AutomaticSpeechRecognitionPipeline(
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor
)

[(1, 50313), (2, 50360), (3, 50364)]


In [ ]:
import gradio as gr

def transribe(audio):
  with torch.cuda.amp.autocast():
    text = pipe(audio, generate_kwargs={'forced_decoder_ids': forced_decoder_ids})['text']
  return text

iface = gr.Interface(
    fn=transribe,
    inputs=gr.Audio("microphone", type='filepath'),
    outputs='text',
    title='PEFT LoRA + INT4 Whisper Large V2 Turbo Nepali',
    description='Realtime demo for NEPALI ASR'
)
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9d30052bf78f000ce0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
